In [1]:
import h5py
import numpy as np
import pandas as pd
import torch
from utils.dataloaders.InstanceDataset import InstanceDataset

In [2]:
event_hdf5_file = "data/instance_samples/Instance_events_counts_10k.hdf5"
event_metadata_file = "data/instance_samples/metadata_Instance_events_10k.csv"
noise_hdf5_file = "data/instance_samples/Instance_noise_1k.hdf5"
noise_metadata_file = "data/instance_samples/metadata_Instance_noise_1k.csv"

In [3]:
event_metadata = pd.read_csv(event_metadata_file, usecols=["trace_name", 'trace_start_time', 'trace_P_arrival_time', 'trace_S_arrival_time', 'trace_P_arrival_sample', 'trace_S_arrival_sample', 'trace_GPD_P_number', 'trace_GPD_S_number', 'trace_EQT_P_number', 'trace_EQT_S_number', 'trace_EQT_number_detections'])

event_metadata.head(10)
# event_metadata.info(verbose=True)
# event_metadata.describe()

# test = event_metadata['trace_S_arrival_sample'] - event_metadata['trace_P_arrival_sample']
# test.describe()







,trace_start_time,trace_P_arrival_time,trace_S_arrival_time,trace_P_arrival_sample,trace_S_arrival_sample,trace_name,trace_GPD_P_number,trace_GPD_S_number,trace_EQT_number_detections,trace_EQT_P_number,trace_EQT_S_number
0,2016-12-04T15:34:43.92Z,2016-12-04T15:35:01.27Z,NaN,1735,NaN,11030611.IV.OFFI..HH,1,2,1.0,1.0,1.0
1,2016-12-04T15:34:48.14Z,2016-12-04T15:35:06.90Z,NaN,1876,NaN,11030611.IV.PIEI..HH,0,2,1.0,1.0,1.0
2,2016-12-04T15:34:48.00Z,2016-12-04T15:35:06.90Z,NaN,1889,NaN,11030611.IV.PIEI..HN,2,1,NaN,NaN,NaN
3,2016-12-04T15:34:43.69Z,2016-12-04T15:35:00.48Z,2016-12-04T15:35:07.35Z,1678,2366.0,11030611.IV.RM33..EH,3,7,2.0,2.0,2.0
4,2016-12-04T15:34:44.38Z,2016-12-04T15:35:00.48Z,2016-12-04T15:35:07.35Z,1610,2297.0,11030611.IV.RM33..HN,1,3,NaN,NaN,NaN
5,2016-12-04T15:34:40.30Z,2016-12-04T15:34:58.10Z,NaN,1780,NaN,11030611.IV.SEF1..HN,1,1,NaN,NaN,NaN
6,2016-12-04T15:34:40.10Z,2016-12-04T15:34:59.82Z,NaN,1971,NaN,11030611.IV.SNTG..HH,1,3,2.0,2.0,2.0
7,2016-12-04T15:34:39.70Z,2016-12-04T15:34:59.82Z,NaN,2012,NaN,11030611.IV.SNTG..HN,0,4,NaN,NaN,NaN
8,2016-12-04T15:34:44.28Z,2016-12-04T15:35:03.91Z,NaN,1962,NaN,11030611.IV.SSFR..HH,1,3,1.0,1.0,1.0
9,2016-12-04T15:34:44.87Z,2016-12-04T15:35:03.91Z,NaN,1903,NaN,11030611.IV.SSFR..HN,0,4,NaN,NaN,NaN


In [14]:


dataset = InstanceDataset(event_hdf5_file, event_metadata_file, noise_hdf5_file, noise_metadata_file, "box_trapezoidal", padding_type="percentage", padding_value=1.5 )

print(len(dataset))
f1, t1 = dataset[3]
print ("=========================")
print((2366.0-1678)*1.5)
print(t1[2366+1031])
print(f1.shape)
# print(f1[0, :10])
# print ("=========================")
# f1, t1 = dataset[10001]
# print(t1)
# print(f1.shape)
# print(f1[0, :10])

11000
1032.0
tensor(0.0010)
torch.Size([3, 12000])


In [ ]:
###############    Les trois canaux concaténés en ligne ################################### 



def load_dataset(filename):
    with h5py.File(filename, 'r') as f:
        if 'data' in f:
            data = f['data']
            if isinstance(data, h5py.Dataset):
                print("Type du dataset 'data':", type(data))
                print("Forme du dataset 'data':", data.shape)
                data_matrix = data[:]  
                print("Matrice de données:", data_matrix)
                return data_matrix
            elif isinstance(data, h5py.Group):
            
                hdf5_data = []
                trace_names = []
                for key in data.keys():
                    dataset = data[key]
                    if isinstance(dataset, h5py.Dataset):
                        # Vérifier la forme des données extraites
                        if dataset.ndim == 2:
                            # Concaténer les trois canaux pour chaque signal
                            concatenated_data = np.concatenate([np.atleast_2d(dataset[i, :]) for i in range(3)], axis=1)
                            hdf5_data.append(concatenated_data)
                            trace_names.append(key)
                        else:
                            print(f"Les données pour la clé '{key}' ne sont pas sous forme de tableau 2D.")
                if hdf5_data:
                    stacked_data_matrix = np.vstack(hdf5_data)
                    df_hdf5 = pd.DataFrame(stacked_data_matrix)
                    df_hdf5['trace_name'] = trace_names
                    return  df_hdf5


# Utilisation de la fonction pour charger et convertir le dataset à partir du fichier HDF5
filename1 = 'Instance_noise_1k.hdf5'
df_noise = load_dataset(filename1)
# Ajouter une colonne 'source_type' avec des valeurs constantes de 1
df_noise['source_type'] = 1


filename2 = 'Instance_events_counts_10k.hdf5'
df_earth = load_dataset(filename2)
len (df_earth)
df_earth['source_type'] = 0
print (df_earth)

In [106]:

############################    récuperation des données dans les csv 


# Spécifier le chemin complet vers le fichier CSV
csv_file1 = 'metadata_Instance_noise_1k.csv'
# Lecture du fichier CSV contenant les informations sur la base de données des signaux
noise_csv = pd.read_csv(csv_file1)

# Spécifier le chemin complet vers le fichier CSV
csv_file2 = 'metadata_Instance_events_10k.csv'
# Lecture du fichier CSV contenant les informations sur la base de données des signaux
earth_csv = pd.read_csv(csv_file2)
print(earth_csv)

      source_id station_network_code station_code  station_location_code  \
0      11030611                   IV         OFFI                    NaN   
1      11030611                   IV         PIEI                    NaN   
2      11030611                   IV         PIEI                    NaN   
3      11030611                   IV         RM33                    NaN   
4      11030611                   IV         RM33                    NaN   
...         ...                  ...          ...                    ...   
9995   11282341                   IV         MPAG                    NaN   
9996   11282341                   MN          BLY                    NaN   
9997   11282341                   MN          BLY                    NaN   
9998   11282341                   MN          CUC                    NaN   
9999   11282341                   MN          CUC                    NaN   

     station_channels  station_latitude_deg  station_longitude_deg  \
0                

In [104]:

################    récuperation des nom de colonnes en commun dans le csv de noise et earthquake

def  common_columns(csv_file1,csv_file2):

    earth_csv = pd.read_csv(csv_file1) 
    noise_csv = pd.read_csv(csv_file2)
    # Obtenir les noms de colonnes en commun
    common_columns = earth_csv.columns.intersection(noise_csv.columns)
    # Afficher les noms de colonnes en commun
    print(common_columns)

# Spécifier le chemin complet vers le fichier CSV
csv_file1 = 'metadata_Instance_events_10k.csv'
# Spécifier le chemin complet vers le fichier CSV
csv_file2 = 'metadata_Instance_noise_1k.csv'

common_columns(csv_file1,csv_file2)


Index(['source_id', 'station_network_code', 'station_code',
       'station_location_code', 'station_channels', 'station_latitude_deg',
       'station_longitude_deg', 'station_elevation_m', 'station_vs_30_mps',
       'station_vs_30_detail', 'trace_start_time', 'trace_dt_s', 'trace_npts',
       'trace_E_median_counts', 'trace_N_median_counts',
       'trace_Z_median_counts', 'trace_E_mean_counts', 'trace_N_mean_counts',
       'trace_Z_mean_counts', 'trace_E_min_counts', 'trace_N_min_counts',
       'trace_Z_min_counts', 'trace_E_max_counts', 'trace_N_max_counts',
       'trace_Z_max_counts', 'trace_E_rms_counts', 'trace_N_rms_counts',
       'trace_Z_rms_counts', 'trace_E_lower_quartile_counts',
       'trace_N_lower_quartile_counts', 'trace_Z_lower_quartile_counts',
       'trace_E_upper_quartile_counts', 'trace_N_upper_quartile_counts',
       'trace_Z_upper_quartile_counts', 'trace_E_spikes', 'trace_N_spikes',
       'trace_Z_spikes', 'trace_name', 'trace_GPD_P_number',
       't